Реальная матрица корреспонденции - trips.csv, ниже используется как матрица trips. Здесь происходит импорт матрицы. 
Сама марица содержит пять столбцов: первый - номера зон i, второй - номера зон j, третий - количество поездок из i в j, четвертый и пятый - время и среднее расстояние на путь из i в j.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

trips=np.array(pd.read_csv(r"C:\users\Vityakter\Gasnikov\trips.csv",encoding='cp866'))
print(trips)

[[  1.    1.   40.   26.    2.2]
 [  1.    2.    4.   49.    8.1]
 [  1.    3.    3.   35.    6.8]
 ...
 [ 22.    1.    1.  140.   84.8]
 [ 22.    2.    3.  120.   72.6]
 [ 22.   22.   18.   25.    1.5]]


T - количество поездок из пункта i в пункт j.
O - количество отправлений из пункта i.
D - количество прибытий в пункт j. 
Ниже производится подсчет перечисленных потоков. 

In [2]:
T=np.zeros([22,22])
O=np.zeros([22])
D=np.zeros([22])
z=0
for i in range(22):
    O[i]=Counter(trips.T[0])[i+1]
    D[i]=Counter(trips.T[1])[i+1]
    tr=np.zeros([22])
    for j in range(z,int(O[i])+z):
        tr[int(trips[j][1])-1]=trips[j][2]
        if j==int(O[i])+z-1:
            z=int(O[i])+z
    T[i]=tr

Вводятся балансировочные коэффициенты A и B для отправлений из i и для прибытий в j соответсвенно. Изначально оба вектора заполняются единицами.

In [3]:
N=4
A=np.zeros([N,22])
B=np.zeros([N,22])
for i in range(22):
    A[0][i]=1
    B[0][i]=1

Матрица издержек подсчитывается из матрицы trips: четвертый и пятый столбцы в ней отвечают за время в пути и расстояние в среднем между пунктами i и j. Ниже c0 соответствует времени поездки, c1 - среднему расстояни между зонами.

In [4]:
c0=np.zeros([22,22])
z=0
for i in range(22):
    tr=np.zeros([22])
    for j in range(z,int(O[i])+z):
        tr[int(trips[j][1])-1]=trips[j][3]
        if j==int(O[i])+z-1:
            z=int(O[i])+z
    c0[i]=tr

c1=np.zeros([22,22])
z=0
for i in range(22):
    tr=np.zeros([22])
    for j in range(z,int(O[i])+z):
        tr[int(trips[j][1])-1]=trips[j][4]
        if j==int(O[i])+z-1:
            z=int(O[i])+z
    c1[i]=tr

Далее вводится гравитационная функция F, как экспонента от издержек (в данном случае под издержками понимается только среднее время поездок между зонами).

In [5]:
def F(a):
    F=np.zeros([22,22])
    for i in range(22):
        for j in range(22):
            F[i][j]=np.exp(-a*c0[i][j])
    return F

Функция ниже реализует метод Синхорна для подсчета балансировочных коэффициентов A и B.

In [10]:
def t(a):
    Ti=np.zeros([N,22,22])
    f=F(a)
    for k in range(N-1):
        for i in range(22):
            if k%2==0:
                B[k+1]=B[k]
                for p in range(22):
                    A[k+1][i]+=(((B[k+1][p]*O[p]*f[i][p])**(-1)))
            else:
                A[k+1]=A[k]
                for v in range(22):
                    B[k+1][i]+=(((A[k+1][p]*D[p]*f[i][p])**(-1)))
            for j in range(22):
                Ti[k][i][j]=(A[k+1][i]*O[i]*D[j]*B[k+1][j])*f[i][j]
    return Ti

Далее, используя введённые функции, необходимо получить оптимальное значение параметра a, введённого в гравитационную функцию F выше (в показателе экспоеннты со знаком минус). Для этого необходимо решить следующую задачу: $$\sum\limits_{i,j} (T_{ij}-T_{ij} (a))^2 \rightarrow \underset{a}min.$$
Первой написана функция, аргумент в которую передаётся значение a, на выходе получаем значение следующей суммы: $\sum\limits_{i,j} (T_{ij}-T_{ij} (a))^2$ 

In [11]:
def R(a):
    R=np.zeros([22,22])
    Tu=t(a)[-2]
    for i in range(22):
        for j in range(22):
            R[i][j]=(T[i][j]-Tu[i][j])**2
        E=np.sum(R[i])
    return np.sum(E)
Nb=np.array([R(0.),R(1.),0])
aleft=0.
aright=1.
while abs(aleft-aright)>10**(-2):
    x=np.arange(aleft,aright,0.001)
    a=(aleft+aright)/2
    Nb[2]=R(a)
    if Nb[2]-Nb[0]>Nb[1]-Nb[0]:
        Nb[1]=R(a)
        aright=a
    else:
        Nb[0]=R(a)
        aleft=a
print(a)

0.0078125


Итак, получаем, что оптимальное значение a=0.00785. Т.о. гравитацонная функция с одним параметром будет иметь такой вид:
$$F=e^{-0.00785c_{ij}}$$